In [67]:
import pandas as pd

In [68]:
maha_pt = pd.read_csv('maha_pt_with_noun_and_verb_count.csv', header=0)
verbs = pd.read_csv('verbs.csv', header=0)

maha_pt_single_verb = maha_pt[maha_pt.countVerbs == 1]
maha_pt_single_verb.head()

,Unnamed: 0,sentID,sent,words,countWords,countVerbs,repatingSeq,repatingNouns
0,0,mbh12_26165.xml,"n1s,i,x6s,n1s,iic,m3s,n1s,n1s,fut a3s","etat,ca,te,toyam,vaqavAmuKa,saYjYitena,pIyamAn...",9,1,"{'n1s': [0, 3, 6, 7]}","{'nominative': [0, 1, 3, 4]}"
2,2,mbh03_14603.xml,"iic,f1s,prf a3s","rAja,putrI,uvAca",3,1,{},{}
5,5,mbh03_14557.xml,"m3p,x1s,ipv[2] a2s,m6s,m1s,m1s,x6s,m1d,i,x6s,m...","tEH,tvam,yAhi,kzatriyasya,eza,vAhaH,mama,vAmyO...",13,1,"{'i': [8, 11], 'x6s': [6, 9], 'ipv[2] a2s': [2...","{'nominative': [2, 3, 4, 5]}"
6,6,mbh12_26053.xml,"m6s,iic,m7s,iic,m7s,n2p,n2p,m1s,iic,i,n2s,ipf[...","tasya,parama,Atmani,ava,sfte,tAni,asTIni,DAtA,...",12,1,"{'m7s': [2, 4], 'n2p': [5, 6], 'iic': [1, 3, 8]}","{'accusative': [3, 4, 6], 'locative': [1, 2]}"
7,7,mbh12_26043.xml,"n2s,pre[1] a2s,x4p,iic,i,n2s,n2s,iic,n2s,i","tat,arhasi,naH,vi,DAtum,SreyaH,yat,an,antaram,iti",10,1,"{'n2s': [0, 5, 6, 8], 'i': [4, 9], 'iic': [3, 7]}","{'accusative': [0, 1, 2, 3]}"


In [69]:
def get_unique_morphids(list_of_ids):
    
    final_list  = []
    for sent in list_of_ids:
        final_list += sent.split(',')
    
    return set(final_list)


unique_morhp_ids = get_unique_morphids(maha_pt_single_verb['sent'].tolist())

In [70]:
### unique morhpids
len(unique_morhp_ids)

237

In [71]:
### get all the verbs
verbs.head()

,SL,Desc,UH
0,pre[1] a1s,*present-indicative class1 active first-person...,{kartari;law;u;eka;parasmEpadI;BvAdi;#;}
1,pre[1] a1d,*present-indicative class1 active first-person...,{kartari;law;u;dvi;parasmEpadI;BvAdi;#;}
2,pre[1] a1p,*present-indicative class1 active first-person...,{kartari;law;u;bahu;parasmEpadI;BvAdi;#;}
3,pre[1] a2s,*present-indicative class1 active second-perso...,{kartari;law;ma;eka;parasmEpadI;BvAdi;#;}
4,pre[1] a2d,*present-indicative class1 active second-perso...,{kartari;law;ma;dvi;parasmEpadI;BvAdi;#;}


In [72]:
verbs_keys = set(verbs['SL'])
morphids_not_verbs = list(unique_morhp_ids - verbs_keys)
len(morphids_not_verbs)

72

In [73]:
def get_verb_index(sent):
    
    verb_value = verbs_keys & set(sent)
#     print(verb_value)
    return sent.index(list(verb_value)[0])

In [74]:
### testing the function
get_verb_index(['n1s','i','x6s','n1s','iic','m3s','n1s','n1s','fut a3s'])

8

In [75]:
sentences = maha_pt_single_verb['sent'].tolist() 
sentences = [sent.split(',') for sent in sentences]

final_df = []
for morhipd in morphids_not_verbs:
    
    before_verb_count = 0
    after_verb_count = 0
    before_and_after_count = 0
    
    for sent in sentences:
        
        if morhipd in sent:
            
            verb_index = get_verb_index(sent)
            morhipd_indexs = [i for i, id in enumerate(sent) if id==morhipd]
            
            temp = map(lambda x: x-verb_index, morhipd_indexs)
            #print temp
            ### based on the pos and negative values 
            ##decide if the morhp id is before or after verb
            neg_count = sum(1 for number in temp if number < 0)
            pos_count = sum(1 for number in temp if number > 0)
            
            if neg_count > 0:
                before_verb_count +=1
            elif pos_count > 0:
                after_verb_count +=1
            
            if pos_count > 0 and neg_count > 0:
                before_and_after_count +=1
            
                
    final_df.append([morhipd, before_verb_count, after_verb_count, before_and_after_count])

In [78]:
final_df = pd.DataFrame(final_df, columns=['morphid', 'before_verb_count', 'after_verb_count', 'both'])

In [80]:
final_df.head()

,morphid,before_verb_count,after_verb_count,both
0,n6s,13,3,1
1,n6p,6,3,0
2,m3p,1,0,0
3,m2p,33,6,1
4,n4s,2,1,0


In [81]:
final_df.to_csv('morphid_before_after_both_count_df.csv', index=False)